# Context Injection - Dynamic Data Flow

This notebook demonstrates how to inject custom context and data into the sandbox execution.

**Use cases:**

- Pass API responses to be processed
- Inject database query results
- Provide user data for transformations
- Supply configuration objects

**Documentation:** [JSR Package](https://jsr.io/@casys/mcp-gateway)

## Setup Sandbox

In [7]:
import { DenoSandboxExecutor } from "jsr:@casys/mcp-gateway";

const sandbox = new DenoSandboxExecutor({
  timeout: 5000,
  memoryLimit: 128,
});

console.log("✅ Sandbox ready");

✅ Sandbox ready


## 1. Basic Context Injection

Inject simple data - each key becomes a variable in scope:

In [8]:
const code = `
// Context variables are injected directly into scope
return {
  receivedName: name,
  receivedAge: age,
  message: \`Hello \${name}, you are \${age} years old\`
};
`;

const result = await sandbox.execute(code, {
  name: "Alice",
  age: 30,
});

console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": {
    "receivedName": "Alice",
    "receivedAge": 30,
    "message": "Hello Alice, you are 30 years old"
  },
  "executionTimeMs": 54.26366399999824
}


## 2. Processing API-like Data

Simulate processing an API response with complex nested data:

In [9]:
const apiResponse = {
  users: [
    { id: 1, name: "Alice", email: "alice@example.com", active: true },
    { id: 2, name: "Bob", email: "bob@example.com", active: false },
    { id: 3, name: "Charlie", email: "charlie@example.com", active: true },
  ],
  metadata: {
    total: 3,
    timestamp: "2025-01-15T10:30:00Z",
  },
};

const code = `
// Filter active users and extract emails
// 'users' and 'metadata' are injected as variables
const activeUsers = users.filter(u => u.active);

return {
  activeCount: activeUsers.length,
  activeEmails: activeUsers.map(u => u.email),
  summary: \`\${activeUsers.length} out of \${metadata.total} users are active\`,
  processedAt: new Date().toISOString()
};
`;

const result = await sandbox.execute(code, apiResponse);
console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": {
    "activeCount": 2,
    "activeEmails": [
      "alice@example.com",
      "charlie@example.com"
    ],
    "summary": "2 out of 3 users are active",
    "processedAt": "2025-11-28T02:04:24.661Z"
  },
  "executionTimeMs": 55.05446699997992
}


## 3. Data Transformation Pipeline

Transform data through multiple operations:

In [10]:
const salesData = {
  transactions: [
    { product: "Laptop", price: 1200, quantity: 2, date: "2025-01-10" },
    { product: "Mouse", price: 25, quantity: 5, date: "2025-01-11" },
    { product: "Keyboard", price: 80, quantity: 3, date: "2025-01-12" },
    { product: "Monitor", price: 300, quantity: 1, date: "2025-01-13" },
  ],
};

const code = `
// Calculate totals and aggregate statistics
// 'transactions' is already injected - use it directly

const totals = transactions.map(t => ({
  ...t,
  total: t.price * t.quantity
}));

const grandTotal = totals.reduce((sum, t) => sum + t.total, 0);
const avgTransaction = grandTotal / totals.length;

// Find best seller
const bestSeller = totals.reduce((best, current) => 
  current.total > best.total ? current : best
);

return {
  totals,
  summary: {
    grandTotal,
    avgTransaction: Math.round(avgTransaction * 100) / 100,
    transactionCount: totals.length,
    bestSeller: {
      product: bestSeller.product,
      revenue: bestSeller.total
    }
  }
};
`;

const result = await sandbox.execute(code, salesData);
console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": {
    "totals": [
      {
        "product": "Laptop",
        "price": 1200,
        "quantity": 2,
        "date": "2025-01-10",
        "total": 2400
      },
      {
        "product": "Mouse",
        "price": 25,
        "quantity": 5,
        "date": "2025-01-11",
        "total": 125
      },
      {
        "product": "Keyboard",
        "price": 80,
        "quantity": 3,
        "date": "2025-01-12",
        "total": 240
      },
      {
        "product": "Monitor",
        "price": 300,
        "quantity": 1,
        "date": "2025-01-13",
        "total": 300
      }
    ],
    "summary": {
      "grandTotal": 3065,
      "avgTransaction": 766.25,
      "transactionCount": 4,
      "bestSeller": {
        "product": "Laptop",
        "revenue": 2400
      }
    }
  },
  "executionTimeMs": 57.37937000000966
}


## 4. Configuration-Driven Execution

Use context to control code behavior dynamically:

In [11]:
const config = {
  data: [10, 25, 30, 15, 40, 5],
  operation: "filter", // Try: 'filter', 'sort', 'transform'
  threshold: 20,
  sortOrder: "desc",
};

const code = `
let result = data;

switch (operation) {
  case 'filter':
    result = result.filter(x => x > threshold);
    break;
  
  case 'sort':
    result = [...result].sort((a, b) => 
      sortOrder === 'desc' ? b - a : a - b
    );
    break;
  
  case 'transform':
    result = result.map(x => x * 2);
    break;
}

return {
  operation,
  original: data,
  result,
  count: result.length
};
`;

const result = await sandbox.execute(code, config);
console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": {
    "operation": "filter",
    "original": [
      10,
      25,
      30,
      15,
      40,
      5
    ],
    "result": [
      25,
      30,
      40
    ],
    "count": 3
  },
  "executionTimeMs": 58.64120600000024
}


## 5. Error Handling with Context

Gracefully handle missing or invalid context:

In [12]:
const code = `
// Validate injected variables before using
if (typeof data === 'undefined') {
  throw new Error("Missing required 'data' variable");
}

if (!Array.isArray(data)) {
  throw new Error("'data' must be an array");
}

return {
  sum: data.reduce((a, b) => a + b, 0),
  count: data.length
};
`;

// Test with invalid context
const result1 = await sandbox.execute(code, { data: "not an array" });
console.log("Invalid context:");
console.log(JSON.stringify(result1, null, 2));

// Test with valid context
const result2 = await sandbox.execute(code, { data: [1, 2, 3, 4, 5] });
console.log("\nValid context:");
console.log(JSON.stringify(result2, null, 2));

Invalid context:
{
  "success": false,
  "error": {
    "type": "RuntimeError",
    "message": "'data' must be an array",
    "stack": "Error: 'data' must be an array\n    at file:<temp-file>:14:9\n    at file:<temp-file>:22:6\n    at file:<temp-file>:45:2"
  },
  "executionTimeMs": 55.06527699998696
}
INFO Sandbox execution succeeded

Valid context:
{
  "success": true,
  "result": {
    "sum": 15,
    "count": 5
  },
  "executionTimeMs": 55.11829700000817
}


## Summary

Context injection enables:

- **Direct variable injection** - Each context key becomes a variable in scope
- **Dynamic data processing** - Transform any JSON data
- **Configuration-driven code** - Control behavior via injected variables
- **API integration** - Process responses safely
- **Validation** - Check variables exist before use

**How it works:**

```typescript
sandbox.execute(code, { name: "Alice", items: [1, 2, 3] });
// Inside sandbox: `name` and `items` are available as variables
```

**Next:** See `03-security-demo.ipynb` for resource limits and permissions